# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

Для начала подключим все необходимые библиотеки:

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import ceil
from numpy.random import RandomState

Загрузим и изучим наши данные

In [2]:
geo_data_1 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_3 = pd.read_csv('/datasets/geo_data_2.csv')

Посмотрим, как выглядят наши данные:

In [3]:
geo_data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


Теперь выведем общую информацию о наших 3х структурах. Посмотрим, есть ли пропуски в данных.

In [4]:
geo_data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
geo_data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
geo_data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Я предлагаю откинуть строку 'id'. Эти признаки помогут обращаться к конкретному человеку, но они помешают нашей модели обучаться.

In [7]:
geo_data_1.drop(['id'], axis=1, inplace=True)
geo_data_2.drop(['id'], axis=1, inplace=True)
geo_data_3.drop(['id'], axis=1, inplace=True)

### Вывод

    Мы загрузили и изучили наши данные. Данных в предобработке не нуждаются, но мы избавились от лишнего столбца 'id'. Переходим к следующему пункту работы.

## Обучение и проверка модели

На данном этапе нам предстоит обучить и проверить модель для каждого региона.

Для начала разобьем наши выборки на тренировочные и валидационные и обучим модели.

In [8]:
# выделим целевой от всех признаков
# для удобства напишем функцию
def pick_features_target(df):
    target = df['product']
    features = df.drop('product', axis=1)
    return features, target

In [9]:
features_1, target_1 = pick_features_target(geo_data_1)
features_2, target_2 = pick_features_target(geo_data_2)
features_3, target_3 = pick_features_target(geo_data_3)

In [10]:
# разделим выборки на тренировачную и валидационную
# для удобства напишем функцию
def pick_train_valid(features, target):
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=0.25, random_state=12345)
    return features_train, features_valid, target_train, target_valid

In [11]:
features_train_1, features_valid_1, target_train_1, target_valid_1 = pick_train_valid(features_1, target_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = pick_train_valid(features_2, target_2)
features_train_3, features_valid_3, target_train_3, target_valid_3 = pick_train_valid(features_3, target_3)

In [12]:
print(features_train_1.shape, features_valid_1.shape, target_train_1.shape, target_valid_1.shape)
print(features_train_2.shape, features_valid_2.shape, target_train_2.shape, target_valid_2.shape)
print(features_train_3.shape, features_valid_3.shape, target_train_3.shape, target_valid_3.shape)

(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)
(75000, 3) (25000, 3) (75000,) (25000,)


Далее мы обучим модель и сделаем предсказания на валидационной выборке для каждого региона. Затем посчитаем RMSE каждой модели.

In [13]:
model_geo_1 = LinearRegression().fit(features_train_1, target_train_1)
model_geo_2 = LinearRegression().fit(features_train_2, target_train_2)
model_geo_3 = LinearRegression().fit(features_train_3, target_train_3)

In [14]:
predicted_valid_1 = model_geo_1.predict(features_valid_1)
predicted_valid_2 = model_geo_2.predict(features_valid_2)
predicted_valid_3 = model_geo_3.predict(features_valid_3)

In [15]:
mse_geo_1 = mean_squared_error(target_valid_1, predicted_valid_1)
mse_geo_2 = mean_squared_error(target_valid_2, predicted_valid_2)
mse_geo_3 = mean_squared_error(target_valid_3, predicted_valid_3)

In [16]:
print(f'RMSE для первого региона = {mse_geo_1 ** 0.5}, средний запас предсказанного сырья = {predicted_valid_1.mean()}')
print(f'RMSE для второго региона = {mse_geo_2 ** 0.5}, средний запас предсказанного сырья = {predicted_valid_2.mean()}')
print(f'RMSE для третьего региона = {mse_geo_3 ** 0.5}, средний запас предсказанного сырья = {predicted_valid_3.mean()}')

RMSE для первого региона = 37.5794217150813, средний запас предсказанного сырья = 92.59256778438038
RMSE для второго региона = 0.893099286775616, средний запас предсказанного сырья = 68.728546895446
RMSE для третьего региона = 40.02970873393434, средний запас предсказанного сырья = 94.96504596800489


Что можно сказать о полученных значениях, 40 единиц это много или мало? Чтобы не сравнивать модель со средним, рассчитаем коэффициент детерминации.

In [17]:
print("R2 для первого региона =", r2_score(target_valid_1, predicted_valid_1))
print("R2 для второго региона =", r2_score(target_valid_2, predicted_valid_2))
print("R2 для третьего региона =", r2_score(target_valid_3, predicted_valid_3))

R2 для первого региона = 0.27994321524487786
R2 для второго региона = 0.9996233978805127
R2 для третьего региона = 0.20524758386040443


Модель для второго региона показала себя лучше всех (почти идеальна). А с остальными можно лишь порадоваться, что значение больше нуля, но до заветной  единицы все ещё далеко.

### Вывод
    
    В этой части работы мы разбили данные на выборки для обучения наших моделей. Вторая модель показала себя почти идеально, а вот с другими двумя ещё нужно работать. 
    На следующем этапе нашего проекта перейдем к расчету прибыли.

## Подготовка к расчёту прибыли

На этом этапе мы подготовим данные для расчета прибыли скважины каждого региона. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним его со средним запасом в каждом регионе.

Для начала рассчитаем достаточный объём сырья для безубыточной работы скважины. Для этого нам необходимо поделить расходы на разработку 200 скважин (10 млрд.) на стоимость одной одной единицы продукта (450 тыс) и поделим его на 200 (количество скважин). Среднее количество сырья, которое должна приносить каждая из 200 скважин в регионе. 

In [18]:
BUDGET  = 10**10
UNIT_COST = 45 * 10**4
min_number_product = (BUDGET  / UNIT_COST)
print('Всего с региона необходимо минимум добыть',ceil(min_number_product), 'тыс. баррелей нефти')
print('В среднем с одной скважины необходимо добыть минимум',ceil(min_number_product / 200), 'тыс. баррелей нефти')

Всего с региона необходимо минимум добыть 22223 тыс. баррелей нефти
В среднем с одной скважины необходимо добыть минимум 112 тыс. баррелей нефти


Теперь мы знаем, чтобы скважины работали без убытка им необходимо принести не меньше 112 тыс. баррелей нефти. Для начала рассчитаем средние значения запасов сырья в каждом регионе и сравним его с необходимым минимумом.

In [19]:
print('Средний запас сырья в первом регионе =', geo_data_1['product'].mean())
print('Средний запас сырья во втором регионе =', geo_data_2['product'].mean())
print('Средний запас сырья в третьем регионе =',geo_data_3['product'].mean())

Средний запас сырья в первом регионе = 92.50000000000001
Средний запас сырья во втором регионе = 68.82500000000002
Средний запас сырья в третьем регионе = 95.00000000000004


Что можно сказать о полученных значения, далеко не каждая скважина нам подойдет, чтобы работать без убытков.

### Вывод

    На данном этапе работы мы подготовили данные для расчета прибыли. Узнали, сколько в среднем должна принести каждая скважина, чтобы окупить затраты на разработку. 
    Так же посмотрели на среднее значения запасов скважин каждого региона. По ним стало понятно, что далеко не каждая скважина нам подойдет, многие придется отбросить. Но ничего, скважин у нас много, так что будет, из чего выбирать.


## Расчёт прибыли и рисков 

    Мы подошли к самому интересному. На этом (последнем) этапе нам предстоит написать алгоритм расчета прибыли по выбранным скважинам и предсказаниям модели. При помощи техники Bootstrap найдем среднее распределение прибыли. Затем, найдем среднюю прибыль и риск убытка. 

Для начала напишем функцию, которая будет считать прибыль по выбранным скважинам 

In [20]:

def calculate_profit(profit_data):
    
    BUDGET  = 10**10
    PRICE_PER_BARREL = 45 * 10**4
    
    # выберем скважины с лучшими предсказаниями
    total_oil = profit_data.sort_values('predict', ascending=0)['target'].head(200).sum()
    
    #Посчитаем какую прибыль принесут эти скважены
    profit = total_oil * PRICE_PER_BARREL - BUDGET 
    return profit

Далее напишем функцию, которая техникой Bootstrap с 1000 выборок будет считать распределение прибыли.

In [21]:
state = RandomState(300520) 

In [22]:
def bootstrap(target, predicted):
    predicted = pd.Series(predicted, index=target.index)
    profit_data = pd.DataFrame(
        dict(target = target, predict = predicted))
    values = []
    
    for i in range(1000):
        subsample_data = profit_data.sample(500, replace=True, random_state=state)
        values.append(calculate_profit(subsample_data))
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean_profit = values.mean()
    risk = sum(values < 0) / len(values) # как я предлагаю рассчитывать риски, но тогда не один регион не подходит условию
    return mean_profit, lower, upper, risk

Найдите среднюю прибыль, 95%-й доверительный интервал и риск убытков для каждого региона

In [23]:
# рассчитываем для 1ого региона
mean_profit_1, lower_1, upper_1, risk_1 = bootstrap(target_valid_1, predicted_valid_1)

In [24]:
print('Значения региона №1')
print(f'Средняя прибыль: {mean_profit_1}')
print(f'95%-й доверительный интервал находится в диапазоне: [{lower_1}; {upper_1}]')
print(f'Риск убытков: {risk_1:.1%}')

Значения региона №1
Средняя прибыль: 385768963.83112484
95%-й доверительный интервал находится в диапазоне: [-137895829.9016504; 930625598.2533405]
Риск убытков: 8.4%


In [25]:
# рассчитываем для 2ого региона
mean_profit_2, lower_2, upper_2, risk_2 = bootstrap(target_valid_2, predicted_valid_2)

In [26]:
print('Значения региона №2')
print(f'Средняя прибыль: {mean_profit_2}')
print(f'95%-й доверительный интервал находится в диапазоне: [{lower_2}; {upper_2}]')
print(f'Риск убытков: {risk_2:.1%}')

Значения региона №2
Средняя прибыль: 450509942.6346684
95%-й доверительный интервал находится в диапазоне: [37365899.88851986; 852378088.6282743]
Риск убытков: 2.0%


In [27]:
# рассчитываем для 3его региона
mean_profit_3, lower_3, upper_3, risk_3 = bootstrap(target_valid_3, predicted_valid_3)

In [28]:
print('Значения региона №3')
print(f'Средняя прибыль: {mean_profit_3}')
print(f'95%-й доверительный интервал находится в диапазоне: [{lower_3}; {upper_3}]')
print(f'Риск убытков: {risk_3:.1%}')

Значения региона №3
Средняя прибыль: 393301341.8945429
95%-й доверительный интервал находится в диапазоне: [-136489823.77742466; 904121251.3918412]
Риск убытков: 6.0%


### Вывод

    На этом этапе мы написали алгоритм, который методом bootstrap брал случайные 500 предсказаний объема запасов нефти. Затем из них выбирал лучшие 200 локаций для станции и считал для них реальные значения запасов нефти. Затем алгоритм рассчитывал прибыль с выбранных станций и рассчитывал 95%-й доверительный интервал.
    Результат: 
* В регионе №1 риск убытков самый большой, 8.4%. В регионе №3 риск убытков уменьшился, 6%, но это значение все ещё не подходит нашей задачи. А вот регион №1 это то, что нам нужно. Риск убытков всего 2%, а нам было необходимо найти регион, в котором риск потерять деньги равен меньше 2.5%.

## Общий Вывод

    В нашей работы мы построили модель, которая предсказывает объём запасов нефти в скважине. Затем с помощью техники bootstrap рассчитывали среднюю прибыль среди выбранных скважин. Результаты показали, что выгодней всего бурить скважины в регионе под номером 2(1), шанс убытка равен 2%, что удволетворяет поставленной задаче. Так как в других регионах шанс убытка более 2.5%, эти регионы, для разработки скважин, мы не рассматриваем.